In [63]:
_PROMPT_TEMPLATE = """
请你提取下面段落的关键词，要求所提取的关键词能够鲜明表示内容性质。所提取的关键词应不少于5个。

待处理内容：

本次股票发行后拟在创业板市场上市，该市场具有较高的投资风险。创业
板公司具有业绩不稳定、经营风险高、退市风险大等特点，投资者面临较大的
市场风险。投资者应充分了解创业板市场的投资风险及本公司所披露的风险因
素，审慎作出投资决定。
华瑞电器股份有限公司
Huarui Electrical Appliance Co., Ltd.
册地址：宁波市鄞州区姜山镇科技园区
首次公开发行股票并在创业板上市
招股意向书
发行人及全体董事、监事、高级管理人员承诺招股意向书不存在虚假记载、
误导性陈述或重大遗漏，并对其真实性、准确性、完整性、及时性承担个别和连
带的法律责任。
发行人及全体董事、监事、高级管理人员、发行人的控股股东、实际控制人
以及保荐人、承销的证券公司承诺因发行人招股意向书及其他信息披露资料有虚
假记载、误导性陈述或者重大遗漏，致使投资者在证券发行和交易中遭受损失的，
将依法赔偿投资者损失。

注意：你的回答不需要分段。你的回答格式应该按照下面的内容，请注意```output 等标记都必须输出，这是我用来提取答案的标记。
除了你的回答之外不要输出多余的文字。

```output
${{"keywords":[你提取的关键词]}}
"""

In [2]:
import dashscope
from dashscope import Generation
from http import HTTPStatus
import random
from CustomLLM import QWENonlie
from Container import g_container
apikey='sk-02b8b1a9175a4cae9bf9571d517d8b81'
llm=QWENonlie.from_name(model_name=Generation.Models.qwen_max,api_key=apikey)
g_container.MODEL = llm

In [6]:
%load_ext autoreload
%autoreload 2
from config import QWEN_CONFIG
from dashscope import TextEmbedding,BatchTextEmbedding
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.embeddings import DashScopeEmbeddings,HuggingFaceEmbeddings
from TxtProcess import faiss_kb_ds
g_container.MODEL = llm
cfg = QWEN_CONFIG(api_key=apikey,
                  embedding_name=BatchTextEmbedding.Models.text_embedding_async_v1)
emb=DashScopeEmbeddings(model=TextEmbedding.Models.text_embedding_v1,
                        dashscope_api_key=apikey)
emb2=HuggingFaceEmbeddings(model_name='../../model/m3e/')
text_ld=TextLoader(file_path='./test_data/03c625c108ac0137f413dfd4136adb55c74b3805.txt')
sp=RecursiveCharacterTextSplitter(
        chunk_size=cfg.chunk_size, chunk_overlap=cfg.chunk_overlap)
docs=text_ld.load_and_split(text_splitter=sp)
fkb=faiss_kb_ds(vs_path=cfg.vec_store_path,kb_path='./test_data/',embs=emb2)
fkb.do_add_doc(docs[:2])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[{'id': 'c71c2fcf-ebf9-4fc8-a058-9b9ac8efc903',
  'meta_data': {'source': './test_data/03c625c108ac0137f413dfd4136adb55c74b3805.txt'}},
 {'id': 'b8025a58-ad5b-4fc9-b320-108b9b43851d',
  'meta_data': {'source': './test_data/03c625c108ac0137f413dfd4136adb55c74b3805.txt'}}]

In [9]:
fkb.do_search("国元证券股份有限公司")

[(Document(page_content='保荐人承诺因其为发行人首次公开发行股票制作、出具的文件有虚假记载、误导性陈述或者重大遗漏，给投资者造成损失的，将先行赔偿投资者损失。发行股票类型：人民币普通股（A股）发行股数：公司拟首次公开发行股票总数不超过 2,167 万股，本次公开发行股份占发行后公司股份总数的比例不低于25%；本次公开发行全部为新股发行，老股东不公开发售股份。每股面值：人民币1.00元每股发行价格：[ ]元预计发行日期：2016年10月13日拟上市证券交易所：深圳证券交易所发行后总股本：不超过8,667万股本次发行前股东所持股份的流通限制及自愿锁定股份的承诺：1、公司控股股东、实际控制人之一、董事长余春明，公司股东、实际控制人之一、董事、总经理余超彪，胡建飞、汪红时、叶松林、刘松林、朱观润等 5名持有公司股份的董事、监事、高级管理人员承诺（其中，持有公司股份的监事仅承诺第（1）项与第（2）项）：（1）自股份公司股票在证券交易所上市交易之日起三十六个月内，不转让或者委托他人管理本次发行前其已持有的股份公司股份，也不由股份公司回购该部分股份。（2）在上述锁定期届满后，若其仍在股份公司任职的，在股份公司任职期间每年转让的股份公司股份不超过其所持有股权总数的25%；自其从股份公司处离职半年内，不转让所持有的股份公司股份；自其申报离任六个月后的十二个月内通过证券交易所挂牌交易出售的股份公司股票数量占其所持有股份公司股票总数的比例不超过50%。（3）所持公司股票在锁定期满后两年内减持的，减持价格不低于发行价；公司上市后 6 个月内如公司股票连续20个交易日的收盘价均低于发行价，或者上市后6个月期末收盘价低于发行价，本人所持公司股票的锁定期限自动延长 6 个月。上述减持价格和股份锁定承诺不因本人职务变更、离职而终止。（前述发行价指公司首次公开发行股票的发行价格，如果公司上市后因派发现金红利、送股、转增股本、增发新股等原因进行除权、除息的，则按照证券交易所的有关规定作除权除息处理。）2、公司股东余春禄、张光秀、陈发娣、余志平承诺：自股份公司股票在证券交易所上市交易之日起三十六个月内不转让或者委托他人管理本次发行前其已持有的股份公司股份，也不由股份公司回购该部分股份。3、除上述股东外，本次发行前的公司其他 67 名自然人股东承诺：自股份公司股票在证券交易所上市交

In [ ]:
ppppp="""
本次股票发行后拟在创业板市场上市，该市场具有较高的投资风险。创业
板公司具有业绩不稳定、经营风险高、退市风险大等特点，投资者面临较大的
市场风险。投资者应充分了解创业板市场的投资风险及本公司所披露的风险因
素，审慎作出投资决定。
华瑞电器股份有限公司
Huarui Electrical Appliance Co., Ltd.
册地址：宁波市鄞州区姜山镇科技园区
首次公开发行股票并在创业板上市
招股意向书
发行人及全体董事、监事、高级管理人员承诺招股意向书不存在虚假记载、
误导性陈述或重大遗漏，并对其真实性、准确性、完整性、及时性承担个别和连
带的法律责任。
发行人及全体董事、监事、高级管理人员、发行人的控股股东、实际控制人
以及保荐人、承销的证券公司承诺因发行人招股意向书及其他信息披露资料有虚
假记载、误导性陈述或者重大遗漏，致使投资者在证券发行和交易中遭受损失的，
将依法赔偿投资者损失。
"""
%load_ext autoreload
%autoreload 2
from Tools.keywords_extra import LLMKeywordsEXChain
testk=LLMKeywordsEXChain.from_llm(llm=llm)
testk._call({"inputs":ppppp})

In [8]:
%load_ext autoreload
%autoreload 2
from TxtProcess import load_dir_txt_process
from config import QWEN_CONFIG
cfg = QWEN_CONFIG(api_key='apikey',
                  embedding_name='text_embedding_async_v1')
load_dir_txt_process(txt_dir='../../bs_challenge_financial_14b_dataset/pdf_txt_file/',
                     pdf_dir='../../bs_challenge_financial_14b_dataset/pdf/',
                     cfg=cfg
                     )


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


  1%|▏         | 1/80 [00:08<11:01,  8.37s/it]

---output
["安徽黄山胶囊股份有限公司"]


  2%|▎         | 2/80 [00:13<08:17,  6.37s/it]

---output
["上海华铭智能终端设备股份有限公司"]


...

---output
["浙江双飞无油轴承股份有限公司"]


 14%|█▍        | 11/80 [00:59<06:01,  5.24s/it]

---output
["上海真兰仪表科技股份有限公司"]


 14%|█▍        | 11/80 [01:01<06:25,  5.59s/it]


KeyboardInterrupt: 

In [11]:
import pdfplumber
with pdfplumber.open('../../bs_challenge_financial_14b_dataset/pdf/398c8e64f18a13e695b5956122ef2f6a6fd3b274.PDF') as p:
    ddd=p.pages[0].extract_text()
ddd

'创业板风险提示：本次股票发行后拟在创业板市场上市，该市场具有较高的投\n资风险。创业板公司具有创新投入大、新旧产业融合成功与否存在不确定性、\n尚处于成长期、经营风险高、业绩不稳定、退市风险高等特点，投资者面临较\n大的市场风险。投资者应充分了解创业板市场的投资风险及本公司所披露的风\n险因素，审慎作出投资决定。\n创业板风险提示：本次股票发行后拟在创业板市场上市，该市场具有较高的投\n资风险。创业板公司具有创新投入大、新旧产业融合成功与否存在不确定性、\n尚处于成长期、经营风险高、业绩不稳定、退市风险高等特点，投资者面临较\n上海真兰仪表科技股份有限公司\n大的市场风险。投资者应充分了解创业板市场的投资风险及本公司所披露的风\n险因素，审慎作出投资决定。\nZenner Metering Technology（Shanghai）Ltd.\n（上海市青浦区盈港东路 6558 号 4 幢）\n首次公开发行股票并在创业板上市\n招股意向书\n保荐机构（主承销商）\n（福州市鼓楼区鼓屏路 27号 1#楼 3层、4 层、5 层）'